## Artificial Neural Network to estimate pCO2sea ##
## Produced by Carvalho, G. T. and Mejia, C., in 2024 ##

### Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from glob import glob 
import pickle
import xarray as xr
import pandas as pd
from datetime import datetime  as dt
import seaborn as sns

In [ ]:
def build_n_model (input_dim, output_dim, hidden_dim, kernel_initializer='he_normal', activation='tanh', verbose=False):

    if np.isscalar(hidden_dim):
        hidden_dim = [hidden_dim]
    
    # create model
    if verbose :
        print('-- Initialize a keras Sequential model')

    model = Sequential()

    # Adding input layer
    if verbose :
        print(f"-- adds input layer: {input_dim} cells")
    model.add(InputLayer(input_shape=(input_dim,)))

    # Adding hidden layers
    for ih,hid_dim in enumerate(hidden_dim):
        if ih == 0 :
            if verbose :
                print(f"-- adds the hidden layer {ih}: {hid_dim} cells, connected to input layer")
        else:
            if verbose :
                print(f"-- adds the hidden layer: {hid_dim} cells, connected to previous one")
        model.add(Dense(units=hid_dim, kernel_initializer=kernel_initializer, activation=activation))

    # Adding output layer
    if verbose :
        print(f"-- adds the output layer: {output_dim} cells, connected to last hidden layer")
    model.add(Dense(units=output_dim, kernel_initializer=kernel_initializer))

    return model

In [ ]:
trajet_file = 'Import database.csv'

In [ ]:
# Loads current measured CSV data into a DataFrame
print(f"Reading boat data from file '{trajet_file}' ...")
socatt_df = pd.read_csv(trajet_file,sep=',',header='infer')

socatt_df

In [ ]:
# After reading CSV file, 'datetime' is readed as type STR. Here we convert column 'datetime' from STR to DATETIME type to deal it as a Time variable
socatt_df['time'] = [dt.strptime(tt,'%d-%m-%Y') for tt in socatt_df['time'].values]

# adds a YEAR column from the 'datetime' one
socatt_df['YEAR'] = socatt_df['time'].dt.year
socatt_df

In [ ]:
# retira los NaN
socatt_df.dropna(inplace=True)
socatt_df

In [ ]:
socatt_df.info()

In [ ]:
socatt_df.describe()

In [ ]:
## Plot data that will be used
socatt_df.hist(bins=40,figsize=(18,8),layout=(4,8));
plt.subplots_adjust(hspace=0.6)
plt.suptitle(f"{socatt_df.shape[0]} datos")

In [ ]:
socatt_df

In [ ]:
# fin de filtrages hay que recalcular el index del DataFrame
socatt_df.reset_index(drop=True,inplace=True) # conserve l'ancien index

In [ ]:
lista_vars_para_nn = ['sss', 'tsm', 'pco2atm','pco2ag']
socatt_df[lista_vars_para_nn].describe()

In [ ]:
#  raw data variable names  |   normalized variable names
# -------------------------   ----------------------------
sst_varlabel = 'tsm';        sst_norm_varlabel  = 'sst_n'
sss_varlabel = 'sss';        sss_norm_varlabel  = 'sal_n'
pco2_varlabel = 'pco2ag';    pco2_norm_varlabel = 'pco2_n'
pco2at_varlabel = 'pco2atm'; pco2at_norm_varlabel = 'pCO2_atm_n'
# other variables
time_varlabel = 'datetime'
lat_varlabel = 'lat'
lon_varlabel = 'lon'
year_varlabel = 'YEAR'
dayofyear_varlabel = 'dayOfYear'

In [ ]:
# normalization of Salinity (Sal or SSS) -> Centre-reduction
# Computes MEAN and STD of Salinity
sss_stat = [socatt_df[sss_varlabel].mean(), socatt_df[sss_varlabel].std()]
# Apply it in order to normalize the values 
socatt_df[sss_norm_varlabel] = (socatt_df[sss_varlabel] - sss_stat[0])/sss_stat[1]

# normalization of Sea Surface Temperature (SST) -> Centre-reduction
sst_stat = [socatt_df[sst_varlabel].mean(), socatt_df[sst_varlabel].std()]
socatt_df[sst_norm_varlabel] = (socatt_df[sst_varlabel] - sst_stat[0])/sst_stat[1]

# normalization of pCO2 sea -> Centre-reduction
pco2_stat = [socatt_df[pco2_varlabel].mean(), socatt_df[pco2_varlabel].std()]
socatt_df[pco2_norm_varlabel] = (socatt_df[pco2_varlabel] - pco2_stat[0])/pco2_stat[1]

# normalization of pCO2 air -> Centre-reduction
pco2at_stat = [socatt_df[pco2at_varlabel].mean(), socatt_df[pco2at_varlabel].std()]
socatt_df[pco2at_norm_varlabel] = (socatt_df[pco2at_varlabel] - pco2at_stat[0])/pco2at_stat[1]

# We save in a dictionary the characteristics of the normalization for each variable
case_dic['sss_stat'] = sss_stat
case_dic['sst_stat'] = sst_stat
case_dic['pco2_stat'] = pco2_stat
case_dic['pco2at_stat'] = pco2_stat

In [ ]:
# using directly the `pairplot()` function from `seaborn` module.
[time_varlabel, lat_varlabel, lon_varlabel, sst_varlabel, sss_varlabel, pco2_varlabel,pco2at_varlabel]

In [ ]:
import tensorflow as tf
from tensorflow.keras import initializers, activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer,Dense
#from keras.utils.vis_utils import plot_model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint #, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras import optimizers
from tensorflow.python.keras.utils.np_utils import to_categorical

In [ ]:
# list of normalized variables composing Input and output
vars_for_model_input = [sss_norm_varlabel,sst_norm_varlabel,pco2at_norm_varlabel] 
vars_for_model_output = [pco2_norm_varlabel]

case_dic['input_vars'] = vars_for_model_input
case_dic['output_vars'] = vars_for_model_output

print(f"\nInput variables :  {vars_for_model_input}")
print(f"Output variables : {vars_for_model_output}")

In [ ]:
socatt_df

In [ ]:
# New DataFrame having case Input and output variables plus Time variable only
data_test_separated = True

if data_test_separated :
    # Datas de Inicio y de Fin del TEST set
    
    test_ident = "Choose file name"; test_time_min = pd.to_datetime('choose a start date, example: 2008-01-01').strftime('%Y-%m-%d'); test_time_max = pd.to_datetime('choose an end date, example: 2010-12-31').strftime('%Y-%m-%d')  # Ano 2010
    #test_ident = "Jun2014'-Feb2015"; test_time_min = pd.to_datetime('2014-06-01').strftime('%Y-%m-%d'); test_time_max = pd.to_datetime('2015-2-28').strftime('%Y-%m-%d')   # junio 2014 a fev 2015
    
    test_df = socatt_df.loc[lambda df: df['time'] >= test_time_min].loc[lambda df: df['time'] <= test_time_max ]   #NON# .reset_index()
    train_df = pd.concat([socatt_df.loc[lambda df: df['time'] < test_time_min], socatt_df.loc[lambda df: df['time'] > test_time_max]],
                          ignore_index=False)
else:
    train_df = socattOK_df[[time_varlabel]+vars_for_model_input+vars_for_model_output]

# compute a random permuted index ou shuffle the lines ofdata
n_patt = train_df.shape[0]

In [ ]:
iperm_randon = np.random.permutation(n_patt)

# create a new DataFrame with random reordered lines of source TRAIN DataFrame 
train_rnd_df = train_df.iloc[iperm_randon,:].copy()

train_rnd_df.reset_index(drop=False,inplace=True) # conserve l'ancien index
train_rnd_df = train_rnd_df.rename(columns={"index": "raw_index"})
#display(train_rnd_df)

if data_test_separated :
    test_seq_df = test_df.copy()
    test_seq_df.reset_index(drop=False,inplace=True) # conserve l'ancien index
    test_seq_df = test_seq_df.rename(columns={"index": "raw_index"})
    #display(test_seq_df)


In [ ]:
if data_test_separated :
    val_split = 0.15

    train_label = 'TRAIN'
    val_label   = 'VAL'
    test_label  = 'TEST'

    n_val = int(val_split * n_patt)
    n_train = n_patt - n_val

    train_rnd_df['dataset'] = [train_label]*n_train + [val_label]*n_val

    n_test = test_df.shape[0]
    
    test_seq_df['dataset'] = [test_label]*n_test

    train_rnd_df = pd.concat((train_rnd_df,test_seq_df), axis=0,
                             ignore_index=True)
else:
    val_split = 0.15
    test_split = 0.15

    train_label = 'TRAIN'
    val_label   = 'VAL'
    test_label   = 'TEST'

    n_val = int(val_split * n_patt)
    n_test = int(test_split * n_patt)
    n_train = n_patt - n_val - n_test

    train_rnd_df['dataset'] = [train_label]*n_train + [val_label]*n_val + [test_label]*n_test

train_rnd_df = train_rnd_df.rename(columns={"index": "raw_index"})


In [ ]:
X_train = train_rnd_df.loc[lambda df: df['dataset'] == train_label,vars_for_model_input].values
Y_train = train_rnd_df.loc[lambda df: df['dataset'] == train_label,vars_for_model_output].values
X_train_raw_index = train_rnd_df.loc[lambda df: df['dataset'] == train_label].index.values

X_val = train_rnd_df.loc[lambda df: df['dataset'] == val_label,vars_for_model_input].values
Y_val = train_rnd_df.loc[lambda df: df['dataset'] == val_label,vars_for_model_output].values
X_val_raw_index = train_rnd_df.loc[lambda df: df['dataset'] == val_label].index.values

if data_test_separated :
    X_test = train_rnd_df.loc[lambda df: df['dataset'] == test_label,vars_for_model_input].values
    Y_test = train_rnd_df.loc[lambda df: df['dataset'] == test_label,vars_for_model_output].values
    X_test_raw_index = train_rnd_df.loc[lambda df: df['dataset'] == test_label].index.values

else:
    X_test = train_rnd_df.loc[lambda df: df['dataset'] == test_label,vars_for_model_input].values
    Y_test = train_rnd_df.loc[lambda df: df['dataset'] == test_label,vars_for_model_output].values
    X_test_raw_index = train_rnd_df.loc[lambda df: df['dataset'] == test_label,'raw_index'].values

print(f"X_train shape: {X_train.shape}\nY_train.shape: {Y_train.shape}")
print(f"X_val shape:   {X_val.shape}\nY_val.shape:   {Y_val.shape}")
print(f"X_test shape:   {X_test.shape}\nY_test.shape:   {Y_test.shape}")


In [ ]:
X_train_raw_index.max(),train_rnd_df.shape

In [ ]:
# compose input and output labels with individual variable names
input_vars_label = '-'.join([v.replace('_n','').upper() for v in vars_for_model_input])
output_vars_label = '-'.join([v.replace('_n','').upper() for v in vars_for_model_output])
#vars_for_model_input+vars_for_model_output

# compose cas name with input and output labels. Its a title or label indentifying the case to be used in figures, etc.
case_label = f"{output_vars_label} from {input_vars_label} [Net1-RndData-wTest]"
# name indentifying converted to label for filename (bo white spacen no '+', no '[' nieder ']' and in lower case. To be used en output and figure filenames
case_label_prnt = case_label.replace(' ','-').replace('+','-').replace('[','').replace(']','').lower()
print(f"Case label ....... {case_label},\nCase_label_prnt .. {case_label_prnt}")

In [ ]:
input_dim  = X_train.shape[1]
output_dim = Y_train.shape[1]
hidden_dim = [10,8,5]

optimizer = 'Adam'
learning_rate = 0.001
model_filename = 'my_model-T{test_ident}-4hc'

# new NN model
model = build_n_model(input_dim, output_dim, hidden_dim)

# Compiling model
print('-- compiling model with loss and optimizer functions')
if optimizer.lower() == 'adam' :
    if learning_rate is None :
        model.compile(loss='mean_squared_error', optimizer='adam')
    else:
        model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(learning_rate=learning_rate))


In [ ]:
save_figs = True
save_nets = True

In [ ]:
# Ploting the network (giving more arguments to the function)
rankdir_value = 'LR' # 'LR' or 'TB'  (left-right or top-bottom)
plot_model_args = { 'show_shapes':False, 'show_layer_names':True, 'rankdir':rankdir_value }
if save_figs :
    figs_filename = f"Net_Model5variaveis"
    figs_filename += f"{model_filename}-{input_dim}-{'-'.join(map(str, hidden_dim))}-{output_dim}"
    figs_filename += f"_case-{case_label_prnt}_model-{rankdir_value}.png"
    figs_filepath = figs_filename
    print("-- saving figure in file ... '{}'".format(figs_filepath))
    plot_model_args['to_file'] = figs_filepath    # ajoute le nom du fichier aux arguments 

plot_model(model, **plot_model_args)

In [ ]:
X_train.shape

In [ ]:
%%time
# the %%time  at first line of a Notebook's cell tells the kernel
# to evaluate execution time of the cell's code

epochs = 50
batch_size = 1024*4

# Training
print(f'Begining training ...')
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=batch_size, epochs=epochs, verbose='auto')


In [ ]:
case_dic['loss'] = history.history['loss']
case_dic['val_loss'] = history.history['val_loss']

# save trained model
if save_nets :
    #model_filepath = os.path.join(nets_dir,model_filename)
    model_filepath = f"{model_filename}_net"
    print(f"-- saving trained model in file '{model_filepath}'...")
    model.save(model_filepath)

    #case_info_file = os.path.join(nets_dir,f"{model_filename}_info.p")
    case_info_file = f"{model_filename}_info.p"
    print(f"-- saving case inrormation model in file '{case_info_file}'...")
    pickle.dump( case_dic, open( case_info_file, "wb" ) )


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')

plt.suptitle(f"{case_label}: Loss curves",y=0.98,size="large");

if save_figs :
    figs_filename = f"Fig_loss-curves"
    figs_filename += f"-{model_filename}-{input_dim}-{'-'.join(map(str, hidden_dim))}-{output_dim}"
    figs_filename += f"_case-{case_label_prnt}"
    #figs_filepath = os.path.join(figs_dir,figs_filename)
    figs_filepath = f"{figs_filename}.png"
    print("-- saving figure in file ... '{}'".format(figs_filepath))
    plt.savefig(figs_filepath)

In [ ]:
# Prediction on Validation data
val_y_pred = model.predict(X_val)

In [ ]:
from scipy.stats.stats import pearsonr

In [ ]:
# denorm y
y_stat_mean, y_stat_std,  = case_dic['pco2_stat']

fig,(ax1,ax2) = plt.subplots(nrows=1,ncols=2,sharex=True,sharey=True,figsize=(16,7))

# Prediction on Validation data
val_y_pred = model.predict(X_val)

y_obs_dnorm = Y_val.flatten()*y_stat_std + y_stat_mean
y_pred_dnorm = val_y_pred.flatten()*y_stat_std + y_stat_mean

rmse = np.sqrt(((y_obs_dnorm - y_pred_dnorm)**2).mean())
corr, _ = pearsonr(y_obs_dnorm, y_pred_dnorm)

ax1.scatter(y_obs_dnorm,y_pred_dnorm,s=2, alpha=0.1)
ax1.set_xlabel('Observado')
ax1.set_ylabel('Calculado')

# linear regression (red line) 
m, b = np.polyfit(y_obs_dnorm,y_pred_dnorm, 1)
x = np.array([100, 500])
ax1.plot(x, m*x + b, c='r')

ax1.axis('image')

lax = ax1.axis()
# plot diagonal
ax1.plot([min((lax[0],lax[2])),min((lax[1],lax[3]))],[min((lax[0],lax[2])),min((lax[1],lax[3]))],ls='-',lw=0.5,c='k')
ax1.axis(lax)

ax1.set_title(f"VAL Obs Vs. Pred (RMS: {rmse:.3f}, Pearson's corr: {corr:.2f} ");

# Prediction on TEST data
test_y_pred = model.predict(X_test)

y_obs_dnorm = Y_test.flatten()*y_stat_std + y_stat_mean
y_pred_dnorm = test_y_pred.flatten()*y_stat_std + y_stat_mean

rmse = np.sqrt(((y_obs_dnorm - y_pred_dnorm)**2).mean())
corr, _ = pearsonr(y_obs_dnorm, y_pred_dnorm)

ax2.scatter(y_obs_dnorm,y_pred_dnorm,s=2, alpha=0.1)
ax2.set_xlabel('Obs')
ax2.set_ylabel('Pred')

# linear regression (red line) 
m, b = np.polyfit(y_obs_dnorm,y_pred_dnorm, 1)
x = np.array([100, 500])
ax2.plot(x, m*x + b, c='r')

ax2.axis('image')

lax = ax2.axis()
# plot diagonal
ax2.plot([min((lax[0],lax[2])),min((lax[1],lax[3]))],[min((lax[0],lax[2])),min((lax[1],lax[3]))],ls='-',lw=0.5,c='k')
ax2.axis(lax)

ax2.set_title(f"TEST Obs Vs. Pred (RMS: {rmse:.3f}, Pearson's corr: {corr:.2f} ");

plt.suptitle(f"{case_label} Obs Vs. Pred ",size='xx-large');
plt.savefig(f'validação_e_teste-{model_filename}.png',dpi=300,tight_layout= True)